<a href="https://colab.research.google.com/github/anjoya/Bot/blob/main/test_chatbotfi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install nltk
!pip install tflearn
import numpy
import tflearn
import tensorflow
import random
import json
import nltk
nltk.download('punkt')
from tensorflow.python.framework import ops
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=09b84c1b5294e4a837d8b4f1432107f27606e88f99871beb1252a8b5b78b0d39
  Stored in directory: /root/.cache/pip/wheels/4a/d5/f8/9585b4a100c0fd73da204ee785457d67c85e1b9050f009a849
Successfully built tflearn


Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
import json
with open('/content/FAQs') as file:

    data = json.load(file)
print (data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'accommodation', 'patterns': ['do you offer accommodation', 'are there boarding facilities?', 'do you have hostels ?'], 'responses': ['No, we currently don’t provide accommodation. However, there are a number of private hostels along Ngong road, which are close to the school.'], 'context_set': ''}, {'tag': 'remote', 'patterns': ['Do you offer remote learning courses?', 'Are there online classes?', 'Can i take the course online?', 'Can i take virtual classes?'], 'responses': ['Moringa School’s remote classes are done live and online; with daily sessions where learners as

In [5]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [6]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [7]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [8]:
training = numpy.array(training)
output = numpy.array(output)

In [9]:
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
model.fit(training, output, n_epoch=1000, batch_size=5, show_metric=True)


Training Step: 1099  | total loss: 0.54936 | time: 0.033s
| Adam | epoch: 100 | loss: 0.54936 - acc: 0.8326 -- iter: 50/54
Training Step: 1100  | total loss: 0.55535 | time: 0.036s
| Adam | epoch: 100 | loss: 0.55535 - acc: 0.8493 -- iter: 54/54
--


In [11]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Hello, I am Kora,the moringa school FAQ bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if results[results_index] >0.3:
            for tg in data["intents"]:
              if tg['tag'] == tag:
                  responses = tg['responses']

            print(random.choice(responses))
        else:
          print('I did not understand, kindly ask another question')

chat()

Hello, I am Kora,the moringa school FAQ bot (type quit to stop)!
You: hi
Good to see you again!
You: free
I did not understand, kindly ask another question
You: how much
The program pacing and course content for both remote and in-person classes are the same. For the in-person classes, students are allowed to be in our premises 2 days per week where they get to meet and interact with their trainers – for the remaining 3 days of the week you will still have live and instructor-led classes but they will be delivered online. For remote classes, all class interactions happen live and in real-time online. Remote classes cost Ksh43,000 and Ksh131,000 for Prep and Core respectively. In-person classes cost Ksh50,000 and Ksh150,000 for Prep and Core respectively.
You: part
I did not understand, kindly ask another question
You: part time
I did not understand, kindly ask another question
You: quit
